In [2]:
import numpy       as np
import pandas      as pd
import seaborn     as sb
import altair      as alt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

### LECTURA DATA

In [3]:
df = pd.read_excel("../macros_AI_sat.xlsx", sheet_name="Hoja2",index_col="Uniprot Code")
df = df.rename_axis(None)
df.rename(columns={"Tipo de Macro": "MacroType"}, inplace=True)
df["Longitud"] = df.Secuencia.str.len()
df["SplittedSeq"] = df.Secuencia.str.split("").apply(np.asarray)#replace(""," ") 
df['UniCode'] = df.index
df.reset_index(drop=True, inplace=True)
df.head()

,MacroType,Secuencia,Longitud,SplittedSeq,UniCode
0,AF-1521-like,MEVLFEAKVGDITLKLAQGDITQYPAKAIVNAANKRLEHGGGVAYA...,192,"[, M, E, V, L, F, E, A, K, V, G, D, I, T, L, K...",O28751
1,AF-1521-like,MEVEVVRELEMDKLKVKLAGGDITKYPAEAIVNAANKYLEHGGGVA...,193,"[, M, E, V, E, V, V, R, E, L, E, M, D, K, L, K...",D3RWS7
2,AF-1521-like,MVVKKFGSVEVVLEKGDITKYPAEAIVNAANKYLEHGGGVALAIAK...,193,"[, M, V, V, K, K, F, G, S, V, E, V, V, L, E, K...",D2RH24
3,AF-1521-like,MKPEVVLRFSGVEVRLVQGDITKYPAEAIVNAANRHLEHGGGVAYA...,194,"[, M, K, P, E, V, V, L, R, F, S, G, V, E, V, R...",A0A0F7ICE9
4,AF-1521-like,MNLTELTFGNLTFKLAQGDITKLPAEAIVNAANKYLEHGGGVALAI...,190,"[, M, N, L, T, E, L, T, F, G, N, L, T, F, K, L...",A0A075LQ95


In [13]:
#https://stackoverflow.com/questions/13166089/string-comparison-in-python-but-not-levenshtein-distance-i-think
#https://stackoverflow.com/questions/37428973/string-distance-matrix-in-python


X = np.asarray(df[["Secuencia"]])
X_name = np.asarray(df["UniCode"])

## Calculo similaritud

In [14]:
import textdistance #levenshtein
tokens_1 = df["Secuencia"][0]
tokens_2 = df["Secuencia"][1]
print(tokens_1)
textdistance.ratcliff_obershelp(tokens_1 , tokens_2)

MEVLFEAKVGDITLKLAQGDITQYPAKAIVNAANKRLEHGGGVAYAIAKACAGDAGLYTEISKKAMREQFGRDYIDHGEVVVTPAMNLEERGIKYVFHTVGPICSGMWSEELKEKLYKAFLGPLEKAEEMGVESIAFPAVSAGIYGCDLEKVVETFLEAVKNFKGSAVKEVALVIYDRKSAEVALKVFERSL


0.7220779220779221

In [15]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist
from Levenshtein import ratio    #Aquí Levenshtein. Probar otras. 

#arr1 = np.array(df["Secuencia"])
#arr2 = np.array(df["Secuencia"])
matrix = cdist(X.reshape(-1, 1), X.reshape(-1, 1), lambda x, y: ratio(x[0], y[0]))
df_max = pd.DataFrame(data=matrix, index=X_name, columns=X_name)
df_max.head()

,O28751,D3RWS7,D2RH24,A0A0F7ICE9,A0A075LQ95,C6A047,Q8U0P9,Q5JER1,O59182,Q9V0Y3,...,P03317,Q8JUX6,P36328,P0C6U8,P0C6X1,P0C6Y5,A0A0P0LKV0,A0A0U2GMU3,B1PHJ4,A0A0U1WHD1
O28751,1.000000,0.748052,0.722078,0.730570,0.685864,0.649215,0.650667,0.672043,0.643979,0.645333,...,0.136784,0.136534,0.137803,0.083953,0.055252,0.055846,0.055212,0.055212,0.086099,0.054230
D3RWS7,0.748052,1.000000,0.725389,0.739018,0.720627,0.689295,0.664894,0.718499,0.663185,0.664894,...,0.138950,0.136483,0.136262,0.084372,0.055532,0.056129,0.055492,0.055492,0.086528,0.054504
D2RH24,0.722078,0.725389,1.000000,0.702842,0.657963,0.621410,0.659574,0.675603,0.637076,0.638298,...,0.137472,0.140232,0.138496,0.084372,0.055532,0.056129,0.055492,0.055492,0.086528,0.054504
A0A0F7ICE9,0.730570,0.739018,0.702842,1.000000,0.677083,0.687500,0.663130,0.705882,0.661458,0.668435,...,0.136683,0.137181,0.136211,0.084790,0.055811,0.056412,0.055771,0.055771,0.086508,0.054779
A0A075LQ95,0.685864,0.720627,0.657963,0.677083,1.000000,0.784211,0.707775,0.756757,0.700000,0.713137,...,0.136145,0.135886,0.136414,0.083115,0.054692,0.055281,0.054653,0.054653,0.085240,0.053680


In [16]:
#añadir al df original
df_new = pd.concat([df, pd.DataFrame(matrix)], axis=1)

In [17]:
df_new.head()

,MacroType,Secuencia,Longitud,SplittedSeq,UniCode,0,1,2,3,4,...,155,156,157,158,159,160,161,162,163,164
0,AF-1521-like,MEVLFEAKVGDITLKLAQGDITQYPAKAIVNAANKRLEHGGGVAYA...,192,"[, M, E, V, L, F, E, A, K, V, G, D, I, T, L, K...",O28751,1.000000,0.748052,0.722078,0.730570,0.685864,...,0.136784,0.136534,0.137803,0.083953,0.055252,0.055846,0.055212,0.055212,0.086099,0.054230
1,AF-1521-like,MEVEVVRELEMDKLKVKLAGGDITKYPAEAIVNAANKYLEHGGGVA...,193,"[, M, E, V, E, V, V, R, E, L, E, M, D, K, L, K...",D3RWS7,0.748052,1.000000,0.725389,0.739018,0.720627,...,0.138950,0.136483,0.136262,0.084372,0.055532,0.056129,0.055492,0.055492,0.086528,0.054504
2,AF-1521-like,MVVKKFGSVEVVLEKGDITKYPAEAIVNAANKYLEHGGGVALAIAK...,193,"[, M, V, V, K, K, F, G, S, V, E, V, V, L, E, K...",D2RH24,0.722078,0.725389,1.000000,0.702842,0.657963,...,0.137472,0.140232,0.138496,0.084372,0.055532,0.056129,0.055492,0.055492,0.086528,0.054504
3,AF-1521-like,MKPEVVLRFSGVEVRLVQGDITKYPAEAIVNAANRHLEHGGGVAYA...,194,"[, M, K, P, E, V, V, L, R, F, S, G, V, E, V, R...",A0A0F7ICE9,0.730570,0.739018,0.702842,1.000000,0.677083,...,0.136683,0.137181,0.136211,0.084790,0.055811,0.056412,0.055771,0.055771,0.086508,0.054779
4,AF-1521-like,MNLTELTFGNLTFKLAQGDITKLPAEAIVNAANKYLEHGGGVALAI...,190,"[, M, N, L, T, E, L, T, F, G, N, L, T, F, K, L...",A0A075LQ95,0.685864,0.720627,0.657963,0.677083,1.000000,...,0.136145,0.135886,0.136414,0.083115,0.054692,0.055281,0.054653,0.054653,0.085240,0.053680


In [ ]:
# ¿Hacer análisis de Ngramas antes o después de este cálculo de similaritud?


#https://stackoverflow.com/questions/13166089/string-comparison-in-python-but-not-levenshtein-distance-i-think

#https://academic.oup.com/peds/article/16/2/103/1566066 (Amino acid grouping!!)

## Clustering

In [ ]:
#jerárquico